In [1]:
import numpy as np
from itertools import count, islice
from math import sqrt, gcd

## Helper-functions

In [2]:
def isPrime(n): # Simple, compact prime check for relatively low numbers
    return n > 1 and all(n%i for i in islice(count(2), int(sqrt(n)-1)))

In [3]:
def non_singular(p,a,b): # Return true if elliptic curve (p,a,b) is non-singular
    c = (4*a**3+27*b**2)%p
    return c != 0

In [4]:
def modInverse(n,p): # Calculate modular inverse
    g = gcd(n,p)
    if g != 1: return -1
    else: return pow(n, p - 2, p) 

In [5]:
def modDivide(m,n,p): # Modular division
    m = m%p
    inv = modInverse(n,p)
    if inv == -1:
        return -1
    else:
        return ((inv*m)%p)

In [6]:
class Point():
    def __init__(self, x,y):
        self.x = x
        self.y = y

## Elliptic curve calculations

In [7]:
# Brute-force approach to finding all points on an elliptic curve.
# This is not practical for large numbers, but works for this purpose
def find_points_on_curve(p,a,b): 
    points = []
    for x in range(p): # for every x in the field, find possible y-values, add the points
        r = (x**3+a*x+b)%p
        y_values = []
        for y in range(p):
            if pow(y,2,p) == r: 
                y_values.append(y)
                
        for y in y_values:
            points.append((x,y))
            
    return points

In [8]:
def slope(p1, p2, p,a,b):
    if p1.x == p2.x and p1.y != p2.y: return -1 # x1=x2 and y1=/=y2 => undefined
    
    if p1.x == p2.x and p1.y == p2.y: # (x1,y1) = (x2,y2)
        return modDivide((3*p1.x**2+a), (2*p1.y), p)
    else: # (x1,y1) =/= (x2,y2)
        return modDivide(p2.y-p1.y, p2.x-p1.x, p)

In [9]:
def multiply_point(point, p, a, b): # Multiply point until it "loops"
    new_point = Point(point.x, point.y)
    print("Multiplying point p = (%d,%d)" % (point.x, point.y))
    k = 2
    while True:
        # find the slope S
        s = slope(new_point, point, p,a,b)
            
        if s == -1:
            print("%d * p = infinity" % k)
            break
        else: #calculate the next point
            x = (s**2-new_point.x-point.x)%p
            y = (s*new_point.x-s*x-new_point.y)%p
            print("%d * p = (%d, %d) + (%d, %d) = (%d, %d)" % 
                  (k,new_point.x,new_point.y, point.x, point.y, x, y))
            new_point = Point(x, y)
        
        k += 1

In [10]:
def analyse_elliptic_curve(p,a,b, calculate_multiplications):
    assert 0 < a < p and 0 < b < p and p > 2, "invalid argument values"
    assert isPrime(p), "p is not prime"
    assert non_singular(p,a,b), "The curve is singular"
    
    #find all points
    points = find_points_on_curve(p,a,b)
    print("Elliptic curve with values p=%d, a=%d, b=%d" % (p,a,b))
    print("All points on the curve: \n" + str(points))
    
    if calculate_multiplications:
        for point in points:
            multiply_point(Point(point[0],point[1]), p, a, b)

In [11]:
analyse_elliptic_curve(17, 3, 6, True)

Elliptic curve with values p=17, a=3, b=6
All points on the curve: 
[(3, 5), (3, 12), (6, 6), (6, 11), (7, 8), (7, 9), (8, 7), (8, 10), (10, 4), (10, 13), (12, 6), (12, 11), (13, 7), (13, 10), (14, 2), (14, 15), (15, 3), (15, 14), (16, 6), (16, 11)]
Multiplying point p = (3,5)
2 * p = (3, 5) + (3, 5) = (3, 12)
3 * p = infinity
Multiplying point p = (3,12)
2 * p = (3, 12) + (3, 12) = (3, 5)
3 * p = infinity
Multiplying point p = (6,6)
2 * p = (6, 6) + (6, 6) = (13, 10)
3 * p = (13, 10) + (6, 6) = (7, 8)
4 * p = (7, 8) + (6, 6) = (8, 7)
5 * p = (8, 7) + (6, 6) = (16, 6)
6 * p = (16, 6) + (6, 6) = (12, 11)
7 * p = (12, 11) + (6, 6) = (3, 5)
8 * p = (3, 5) + (6, 6) = (10, 4)
9 * p = (10, 4) + (6, 6) = (14, 15)
10 * p = (14, 15) + (6, 6) = (15, 3)
11 * p = (15, 3) + (6, 6) = (15, 14)
12 * p = (15, 14) + (6, 6) = (14, 2)
13 * p = (14, 2) + (6, 6) = (10, 13)
14 * p = (10, 13) + (6, 6) = (3, 12)
15 * p = (3, 12) + (6, 6) = (12, 6)
16 * p = (12, 6) + (6, 6) = (16, 11)
17 * p = (16, 11) + (6, 6)